In [1]:
from datasets import load_dataset
import pandas as pd
import numpy as np
from itertools import islice

c:\Users\georg\anaconda3\envs\TextMining310\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
c:\Users\georg\anaconda3\envs\TextMining310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
book_iter = load_dataset("bookcorpus", "plain_text", split='train', streaming=True, trust_remote_code=True)
book_sample = list(islice(book_iter, 5000))

In [3]:
splits = {'train': 'train.csv', 'test': 'test.csv'}
sports_df = pd.read_csv("hf://datasets/okite97/news-data/" + splits["train"])

In [4]:
movie_df = pd.read_csv("datasets-to-merge/IMDB Dataset.csv")
movie_sample = movie_df.sample(n=5000, random_state=42).reset_index(drop=True)

In [5]:
book_df = pd.DataFrame(book_sample)

In [6]:
movie_sample = movie_sample.rename(columns={'review': 'text'})
movie_sample['topic'] = 'movie'
movie_sample = movie_sample.drop(columns=['sentiment'])
print(movie_sample.head())

                                                text  topic
0  I really liked this Summerslam due to the look...  movie
1  Not many television shows appeal to quite as m...  movie
2  The film quickly gets to a major chase scene w...  movie
3  Jane Austen would definitely approve of this o...  movie
4  Expectations were somewhat high for me when I ...  movie


In [7]:
book_df['topic'] = 'book'
print(book_df.head())

                                                text topic
0  usually , he would be tearing around the livin...  book
1  but just one look at a minion sent him practic...  book
2  that had been megan 's plan when she got him d...  book
3  he 'd seen the movie almost by mistake , consi...  book
4  she liked to think being surrounded by adults ...  book


In [8]:
sports_df = sports_df.drop(columns=['Excerpt'])
sports_df = sports_df.rename(columns={'Title': 'text', 'Category': 'topic'})
sports_df['topic'] = 'sports'
sports_df = sports_df.dropna(subset=['text'])
sports_df = sports_df.head(5000)
print(sports_df.head())

                                                text   topic
0  Uefa Opens Proceedings against Barcelona, Juve...  sports
1  Amazon Blames Inflation as It Increases Cost o...  sports
2  Nigeria’s Parliament Passes Amended Electoral ...  sports
3  Nigeria: Lagos Governor Tests Positive for Cov...  sports
4  South Africa Calls For Calm as Electoral Refor...  sports


In [9]:
missing_text_rows = sports_df[sports_df['text'].isnull()]
print(missing_text_rows)

Empty DataFrame
Columns: [text, topic]
Index: []


In [10]:
print(len(movie_sample), len(book_df), len(sports_df))

5000 5000 4685


In [11]:
combined_df = pd.concat([book_df, sports_df, movie_sample], ignore_index=True)

In [12]:
combined_df.to_csv("topic-analysis-dataset.csv", index=False)

In [13]:
df = pd.read_csv("topic-analysis-dataset.csv")
unique_topics = df['topic'].unique()
print(f"Unique topics: {unique_topics}")
print(f"Number of unique topics: {len(unique_topics)}")
topic_counts = df['topic'].value_counts()
print(topic_counts)

Unique topics: ['book' 'sports' 'movie']
Number of unique topics: 3
topic
book      5000
movie     5000
sports    4685
Name: count, dtype: int64
